<a href="https://colab.research.google.com/github/KSY1526/myblog/blob/master/_notebooks/2021-12-21-Do_natural_language2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "[Do it 자연어] 2. 숫자 세계로 떠난 자연어"
- author: Seong Yeon Kim 
- categories: [book, jupyter, Do it, natural language, BERT, self-attention, tokenizer]

# 셀프 어텐선 동작원리

In [4]:
import torch
x = torch.tensor([
    [1.0, 0.0, 1.0, 0.0],
    [0.0, 2.0, 0.0, 2.0],
    [1.0, 1.0, 1.0, 1.0],
])

w_query = torch.tensor([
    [1.0, 0.0, 1.0],
    [1.0, 0.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 1.0],
])

w_key = torch.tensor([
    [0.0, 0.0, 1.0],
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 0.0],
    [1.0, 1.0, 0.0],
])

w_value = torch.tensor([
    [0.0, 2.0, 0.0],
    [0.0, 3.0, 0.0],
    [1.0, 0.0, 3.0],
    [1.0, 1.0, 0.0],
])

변수를 정의합니다.

In [7]:
keys = torch.matmul(x, w_key)
querys = torch.matmul(x, w_query)
values = torch.matmul(x, w_value)

쿼리, 키, 벨류를 만듭니다.

In [10]:
attn_scores = torch.matmul(querys, keys.T)
attn_scores

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

어텐션 스코어를 만듭니다.

In [11]:
import numpy as np
from torch.nn.functional import softmax
key_dim_sqrt = np.sqrt(keys.shape[-1])
attn_probs = softmax(attn_scores / key_dim_sqrt, dim = -1)
attn_probs

tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])

소프트맥스 확률값을 만듭니다.

In [12]:
weighted_values = torch.matmul(attn_probs, values)
weighted_values

tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])

소프트맥스 확률과 밸류를 가중합 하였습니다. 셀프 어텐션에 최종 출력값 입니다.

셀프 어텐션은 가중치 행렬(w_..) 3개를 학습 대상으로 생각하고 태스크를 잘 수행하는 방향으로 업데이트 됩니다.

# 피드포워드 뉴럴 네트워크 계산 예시

In [13]:
import torch

x = torch.tensor([2,1])
w1 = torch.tensor([[3,2,-4],[2,-3,1]])
b1 = 1
w2 = torch.tensor([[-1,1],[1,2],[3,1]])
b2 = -1

변수를 입력합니다.

In [14]:
h_preact = torch.matmul(x, w1) + b1
h = torch.nn.functional.relu(h_preact)
y = torch.matmul(h, w2) + b2

y

tensor([-8, 12])

결과 값 입니다. 여기서 w1, w2, b1, b2가 학습 대상이 됩니다.

학습 대상은 태스크를 잘 수행하는 방향으로 업데이트 됩니다.

In [18]:
m = torch.nn.Dropout(p = 0.2)
input = torch.randn(1,10)
output = m(input)
print(input)
print(output)

tensor([[ 0.3678,  1.6664,  1.6091,  0.1445,  0.9486,  2.0537,  2.4539,  1.4420,
          0.9701, -0.2877]])
tensor([[0.0000, 2.0830, 2.0113, 0.1806, 1.1858, 2.5672, 0.0000, 1.8025, 0.0000,
         -0.0000]])


간단한 드롭다웃 예제입니다. p 확률 만큼 뉴련을 0으로 대치해 계산에서 제외합니다.

# 문장을 벡터로 변환하기

In [19]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 806 kB 15.9 MB/s 
     |████████████████████████████████| 57 kB 6.4 MB/s 
     |████████████████████████████████| 2.8 MB 37.5 MB/s 
     |████████████████████████████████| 636 kB 56.2 MB/s 
     |████████████████████████████████| 829 kB 59.2 MB/s 
     |████████████████████████████████| 132 kB 52.0 MB/s 
     |████████████████████████████████| 332 kB 64.2 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
     |████████████████████████████████| 61 kB 671 kB/s 
     |████████████████████████████████| 3.3 MB 48.7 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████████| 96 kB 6.6 MB/s 
     |████████████████████████████████| 271 kB 69.4 MB/s 
     |████████████████████████████████| 160 kB 57.9 MB/s 
     |████████████████████████████████| 192 kB 71.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 

In [20]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    'beomi/kcbert-base',
    do_lower_case = False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BERT(kcbert-base) 모델이 쓰는 토크나이저를 선언합니다. 

In [21]:
from transformers import BertConfig, BertModel
pretrained_model_config = BertConfig.from_pretrained(
    'beomi/kcbert-base'
)
model = BertModel.from_pretrained(
    'beomi/kcbert-base',
    config = pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT(kcbert-base) 모델을 읽어들입니다.

In [22]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

pretrained_model_config은 BERT 모델을 프리트레인 할때 설정했던 내용이 있습니다.

블록 수는 12개, 헤드 수는 12개, 어휘 집합 크기는 3만개 입니다.

In [24]:
sentences = ['안녕하세요', '하이!']
features = tokenizer(
    sentences,
    max_length = 10,
    padding = 'max_length',
    truncation = True,
)
features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

BERT 모델의 입력값을 만듭니다. 앞서 배운 BERT 모델과 같이 3개의 변수가 나옵니다.

In [25]:
features = {k : torch.tensor(v) for k, v in features.items()}

피처를 파이토치에 넣기 위해선 자료형이 텐서(tensor)이여야 하기 때문에 자료형을 변경했습니다.

In [27]:
outputs = model(**features)
outputs

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
                                                        [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
                                                        [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
                                                        ...,
                                                        [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
                                                        [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
                                                        [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],
                                               
                                                       [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.403

BERT 모델에 features를 적용했습니다. 두 개의 출력물 last_hidden_state, pooler_output이 나옵니다.

전자를 단어수준 임베딩, 후자를 문장수준 임베딩이라고 부릅니다.